# IDEA
* Multi-Task RL, e.g., optimize both drawdown and returns
* 

# Environment
## Data Vendor
* Quandl
* CSI
* CRSP
* ORATS.com(Option)
* 

# General Algorithm
## Predict FX through Order Flow
* Blog, http://epchan.blogspot.com/
* Use last 20 days' volume to make histgram of trade flows
* If the volume falls in top(bottom), long(short).

## Vol Targeting and Trend Following
* Blog,  https://qoppac.blogspot.com/2018/07/vol-targeting-and-trend-following.html
* Scale the position according to the volatility
* Better Kurtosis and Sharpe Ratio while worse Skew

# BVC
* Paper, https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1989555&download=yes
* Get rid of necessity of tick data
* Dark pool order makes volume inaccurate
* Fixed-volume bar has more Gaussian distribution than fixed-time bars for log returns

# Execution
## Hide-and-Light order
* Order to sell stocks has to be routed the market with NBBO(National Best Bid Offer)
* Avoid being changed fee to execute an order at market price
* Set the selling price lower till the target market raise the best offer
* Earn the liquidity rebate
* Get the chance to place the order at the top of the queue of the target price

## ISO(Intermediate Sweep Order)
* Need not be routed to a market with NBBO
* Faster execution due to non-necessity to check the NBO

# Supervised Learning

## Automated Bitcoin Trading via Machine Learning Algorithms
* Paper, http://cs229.stanford.edu/proj2014/Isaac%20Madan,%20Shaurya%20Saluja,%20Aojia%20Zhao,Automated%20Bitcoin%20Trading%20via%20Machine%20Learning%20Algorithms.pdf
* Features, 5 years
    * Average Confirmation Time
    * Block Size
    * Cost per transaction percent
    * Difficulty to find a new block
    * Hash Rate
    * Market Capitalization
    * Miners Revenue
    * Number of Orphaned Blocks
    * Number of transactions per Blocks
    * Number of unique transactions per day
    * Number of unique address
    * Total Bitcoins
    * Transaction fees total per day
    * Trade Volume
    * Transaction to trade ratio
* Data Resources
    * Blockchain Info, https://www.blockchain.com/explorer
    * OKCoin
    * Coinbase, https://developers.coinbase.com/
* Binary Classification
* Combine three different time scale models with linear combination

## Comparative Automated Bitcoin Trading Strategies
* Paper, http://cs229.stanford.edu/proj2016/report/MumfordHegazy-ComparitiveAutomatedBitcoinTradingStrategies-report.pdf
* Intraday Frequency
* Prices are smoothed out by locally weighted linear regression
* Betting Size is determined by Classification Confidence

## Anticipating cryptocurrency prices using machine learning
* Daily Frequency
* Currencies with more than $10^5$ USD volume each day
* Currencies are characterized by the followings:
    * Price
    * Market Capitalization
    * Market Share
    * Rank based on market capitalization
    * Volume
    * Age, life of the currency in days

* Three methods
    1. XGBoost1
        - Target: ROI across different currencies
        - Featrues: price, market cap, market share, rank, volume, and ROI
    2. XGBoost2
        - Target: ROI of target currency
        - Features: Use features of all currencies
    3. LSTM
        - Different Model for each currency
        - Feature: Only ROI of given currency

# Reinforcement Learning
## Deep Reinforcement Learning in High Frequency Trading
* https://arxiv.org/pdf/1809.01506.pdf
* Ensemble three one-vs-one MLP, each of which predict binary labels
* Ensemble weight keeps updating through Reinforcement Learning
* 500 trailing tick history and label for 100 forward
* MLP has (10, 10) hidden layers
* Accuracy will be around 70 %
* Only when confidence level is over the threshold, execute trading
* The perticipant percentage is around 10%

## A Deep Reinforcement Learning Framework for the Financial Portfolio Management Problem
* https://arxiv.org/pdf/1706.10059.pdf
* Use Close, High, and Low as input
* Use replay memory and portfolio vector memory for training
* Use 50 trailing history

## Deep Hedging
* https://arxiv.org/pdf/1802.03042.pdf
* Find the hedging strategy through Neural Network
* Optimize convex measure
* Show the experiments over numerically sampled Heston model
* Use (2d, d+15, d+ 15, d) MLP Model, where d is the number of assets

## Agent Inspired Trading Using Recurrent Reinforcement Learning and LSTM Neural Networks
* https://arxiv.org/pdf/1707.07338.pdf
* Direct reinforcement(Not required value function)
* Recurrent Reinforcement Learning using LSTM or RNN
* Optimize Sharp Ratio or Downside Deviation
* Input:
    * Trailing history of price difference
    * Previous position
    * Bias parameter
* Previous position is fed into the input of the output layer
* Validation is done on a single period, which is not trustable


## QLBS: Q-Learner in the Black-Scholes(-Merton) Worlds
* https://arxiv.org/pdf/1712.04609.pdf
* Derive pricing through Q-Learning format